In [ ]:
# パッケージのクローンとセットアップ
!git clone https://github.com/VOICEVOX/voicevox_core -b 0.13.2
%cd voicevox_core/

In [ ]:
# 環境構築

# ONNX Runtimeのダウンロード
!mkdir onnxruntime
!wget https://github.com/microsoft/onnxruntime/releases/download/v1.10.0/onnxruntime-linux-x64-gpu-1.10.0.tgz
!tar xf onnxruntime-linux-x64-gpu-1.10.0.tgz -C onnxruntime --strip=1
!rm onnxruntime-linux-x64-gpu-1.10.0.tgz
# コアライブラリのダウンロード
!mkdir release
!wget https://github.com/VOICEVOX/voicevox_core/releases/download/0.13.2/voicevox_core-linux-x64-gpu-0.13.2.zip
!unzip -qj voicevox_core-linux-x64-gpu-0.13.2.zip -d release
!rm voicevox_core-linux-x64-gpu-0.13.2.zip
# 配置
!mkdir -p core/lib
!cp onnxruntime/lib/* core/lib
!cp release/* core/lib
# 辞書データダウンロード
!wget http://downloads.sourceforge.net/open-jtalk/open_jtalk_dic_utf_8-1.11.tar.gz
!tar xf open_jtalk_dic_utf_8-1.11.tar.gz
!rm open_jtalk_dic_utf_8-1.11.tar.gz

In [ ]:
!pip install -qqU .

In [ ]:
!pip install aiohttp pytchat numpy sounddevice nest_asyncio openai ipython cohere tiktoken nest_asyncio

In [ ]:
import core
import asyncio
import pytchat
import queue
import random
import openai
import IPython.display
from IPython.display import display, HTML, Javascript
from ipywidgets import widgets
from base64 import b64decode
from google.colab import output
from google.colab import userdata
import nest_asyncio
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/My Drive/AItuber/character.txt'
with open(file_path) as f:
	character = f.read()

In [ ]:
def create_character(character):
    return [{"role": "system", "content": character}]
messages = create_character(character)

In [ ]:
import core

def initialize_core(use_gpu: bool, cpu_num_threads: int, openjtalk_dict: str):
    # コアの初期化
    core.initialize(use_gpu, cpu_num_threads)

    # openjtalk辞書のロード
    core.voicevox_load_openjtalk_dict(openjtalk_dict)

initialize_core(use_gpu=True, cpu_num_threads=0, openjtalk_dict="open_jtalk_dic_utf_8-1.11")

In [ ]:
def tts_and_save(text: str, speaker_id: int):
    wavefmt = core.voicevox_tts(text, speaker_id)
    if not wavefmt:
        return False
    try:
        with open("data.wav", "wb") as f:
            f.write(wavefmt)
        return True
    except IOError as e:
        print(f"ファイル保存時のエラー: {e}")
        return False

def text_to_speech(text, speaker_id):
    if tts_and_save(text, speaker_id):
        file_path = 'data.wav'
        display(IPython.display.Audio(file_path, autoplay=True))
    else:
        print("TTSの実行に失敗しました")


In [ ]:
deepl_api_key = userdata.get('DEEPL_API_KEY')

def translate_to_english(text):
    params = {
                "auth_key": deepl_api_key,
                "text": text,
                "source_lang": 'JA',
                "target_lang": 'EN'
            }
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params)
    result = request.json()
    return result["translations"][0]["text"]

In [ ]:
def update_text_in_frame(text):
    # テキストを更新するJavaScriptコード
    javascript = f'''
    const container = document.getElementById('text-container');
    container.innerHTML = `
      <link rel="preconnect" href="https://fonts.googleapis.com">
      <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
      <link href="https://fonts.googleapis.com/css2?family=Yusei+Magic&display=swap" rel="stylesheet">
      <style>
      h1 {{
        font-family: 'Yusei Magic', sans-serif;
        color: pink;
      }}
      </style>
      <h1>{text}</h1>
    `;
    '''
    display(Javascript(javascript))

In [ ]:
openai.api_key = userdata.get('OPENAI_API_KEY')

def fetch_AI_response(messages):
    response =  openai.chat.completions.create(
                    model="gpt-3.5-turbo-1106",
                    max_tokens=50,
                    temperature=0.2,
                    messages=messages)
    return response

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(e.srcElement.result);
  reader.readAsDataURL(blob);
});

var recorder, chunks, stream;

window.startRecording = async () => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  recorder = new MediaRecorder(stream);
  chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
};

window.stopRecording = () => {
  recorder.stop();
  stream.getTracks().forEach(track => track.stop());
  return new Promise(resolve => {
    recorder.onstop = async () => {
      blob = new Blob(chunks);
      text = await b2text(blob);
      resolve(text);
    };
  });
};
"""

start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")
output_area = widgets.Output()

def on_start_clicked(b):
    output_area.clear_output()
    with output_area:
        print("Recording started...")
    output.eval_js('startRecording()')

def on_stop_clicked_sync(b):
    asyncio.create_task(on_stop_clicked(b))

def on_stop_clicked(b):
    with output_area:
        print("Recording stopped, transcribing...")
        audio_data = output.eval_js('stopRecording()')
        transcribed_text = speech_to_text(audio_data)
        messages.append({"role": "user", "content": transcribed_text})
        response = fetch_AI_response(messages)
        if response is not None:
                    resp = response.choices[0].message.content.strip()
                    print(resp)
                    text_to_speech(resp, speaker_id=20)
                    text = translate_to_english(resp)
                    update_text_in_frame(text)
                    messages.append({"role": "assistant", "content": resp})

def speech_to_text(audio_data, model='whisper-1', language='ja'):
    b = b64decode(audio_data.split(',')[1])

    with open('tmp.wav', 'wb') as fw:
        fw.write(b)

    with open('tmp.wav', "rb") as fr:
        transcription = openai.audio.transcriptions.create(
            model=model,
            file=fr,
            language=language,
            response_format="text"
        )
        return transcription

In [ ]:
comments_queue = asyncio.Queue()
is_chat_ended = asyncio.Event()

async def get_chat(video_id, organizer_name="V太郎", end_message="終わり"):
    livechat = pytchat.create(video_id=video_id)
    print(f"ライブチャットを開始しました: {livechat.is_alive()}")
    while livechat.is_alive():
        try:
            chatdata = await asyncio.to_thread(livechat.get)
            if chatdata:
                print(f"取得したチャットデータ: {chatdata.items}")
                for c in chatdata.items:
                    print(f"キューに追加するメッセージ: {c.author.name}: {c.message}")
                    await comments_queue.put(c)
                    if c.author.name == organizer_name and c.message == end_message:
                        print("チャンネル主催者からの終了メッセージが検出されました。プロセスを終了します。")
                        is_chat_ended.set()
                        return  # ループを抜け、関数を終了
            else:
                print("取得したチャットデータが空です。")
        except Exception as e:
            print(f"チャットデータの取得中にエラーが発生しました: {e}")
        await asyncio.sleep(5)


In [ ]:
async def drain_queue(queue, timeout):
    items = []
    try:
        # キューからアイテムを取得するのを最大 timeout 秒間待機する
        while True:
            item = await asyncio.wait_for(queue.get(), timeout=timeout)
            items.append(item)
    except asyncio.TimeoutError:
        # タイムアウトが発生したら、これ以上アイテムは待たずに処理を続行
        pass
    return items

async def process_chat_message(queue,messages,keywords):
    COMMENT_THRESHOLD = 1  # 閾値を設定（1以上であれば処理を変更）
    TIMEOUT = 30  # タイムアウト時間を設定（秒）
    while not is_chat_ended.is_set():
        comments = await drain_queue(queue, TIMEOUT)
        if not comments:
            # タイムアウト内にコメントが一つもなかった場合の処理
            ("コメントが一定時間ありませんでした。一人で話して続けて...")
            chat_message = "コメントがなかったのであなたは新しい話をするか前回話をしていた続きを離します。いきなり話し始めてください"
            # 一人で話す処理をここに追加
        elif len(comments) > COMMENT_THRESHOLD:
            # コメントが閾値より多い場合の処理
            print(f"多数のコメントがあります: {comments}")
            relevant_comments = [comment for comment in comments if any(keyword in comment.message for keyword in keywords)]
            if relevant_comments:
                # キーワードに関連するコメントがある場合
                chosen_comment = random.choice(relevant_comments)
                print(f"選択された関連するコメント: {chosen_comment.author.name}: {chosen_comment.message}")
            else:
                # 関連するコメントがない場合、ランダムに選択
                chosen_comment = random.choice(comments)
                print(f"ランダムに選択されたコメント: {chosen_comment.author.name}: {chosen_comment.message}")
            chat_message = chosen_comment
        else:
            # それ以外（コメントが一つだけある場合）の処理
            print(f"取得したコメント: {comments}")
            # コメントが一つだけの場合の処理をここに追加
            messages.append({"role": "user", "content": chat_message})
        response = await asyncio.to_thread(
        fetch_AI_response,messages)
        if response is not None:
                resp = response.choices[0].message.content.strip()
                print(resp)
                update_text_in_frame(resp)
                text_to_speech(resp, speaker_id=20)
                text = translate_to_english(resp)
                messages.append({"role": "assistant", "content": resp})
        await asyncio.sleep(TIMEOUT)


In [ ]:
nest_asyncio.apply()
display(Javascript(RECORD))
async def main():
  select_mode = input("おしゃべりモード：１, 配信モード：2")
  html_frame = HTML('''
  <div id="text-container" style="border:1px solid #ccc; padding:10px; width:1000px; height:200px; overflow:auto;">
  ここにテキストが表示されます。
  </div>
  ''')
  display(html_frame)
  if select_mode == "1":
    start_button.on_click(on_start_clicked)
    stop_button.on_click(on_stop_clicked)
    display(widgets.HBox([start_button, stop_button]), output_area)

  if select_mode =="2":
    video_id = input("video_idを入力して")
    important_words = input("重要な単語をカンマ区切りで教えてください")
    collection_of_important_words = important_words.split(", ")
    streaming = await asyncio.gather(
        asyncio.create_task(get_chat(video_id)),
        asyncio.create_task(process_chat_message(comments_queue,messages,collection_of_important_words))
    )

asyncio.run(main())
